<a href="https://colab.research.google.com/github/phys-ken/ypc_database/blob/master/ypc_database_tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YPCの例会アルバムをスクレイピングする
* [YPC過去の例会アルバム](https://www2.hamajima.co.jp/~tenjin/ypc/ypcalbum.html)にアクセスして、全部のリンク先の記事にアクセスして、文書データのみをcsvで出力します。大体、20分くらいかかります。

### 技術的な解説
* pythonのBeautifulsoup4でスクレイピングし、改行コード等はテキスト処理で切り取っています。
* pandasにデータフレームとして呼び込み、行列を整理し出力します。
* [こちら](https://phys-ken.github.io/ypc_database/README.html)のページで、csvを検索することが可能になっています。

In [1]:
#!pip install chardet
#文字化け対策(https://orangain.hatenablog.com/entry/encoding-in-requests-and-beautiful-soup)
#!pip install tqdm

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import tqdm

In [3]:
url = "https://www2.hamajima.co.jp/~tenjin/ypc/ypcalbum.html"
r = requests.get(url)
time.sleep(3)

In [4]:
soup = BeautifulSoup(r.content, "html.parser")

In [5]:
titles_html = soup.find_all("dt")
titles = []
for title in titles_html:
    titles.append(title.find("a"))

In [6]:
top_url = "https://www2.hamajima.co.jp/~tenjin/ypc/"

url_list = []
for title in titles:
    url_list.append([title.get_text(),top_url + title.get("href")])

In [7]:
print("全部で" + str(len(url_list)) + "回処理を行います、")

for i in tqdm.tqdm(range(len(url_list))):
    r = requests.get(url_list[i][1])
    time.sleep(3)
    soup =  BeautifulSoup(r.content, "html.parser")

    ps =soup.find_all("p")

    strip_p = []
    for p in ps:
        if p.get("align") == None:
            strip_p.append(p)

    theme_list = []
    counter = -1

    for p in strip_p:
        try:
            if p.find("font").get("size") == ("+3"):
                counter += 1
                theme_list.append([p.find("font").get_text()])
                theme_list[counter].append("")
                theme_list[counter][1] += p.get_text().strip().replace( '\n' , '.' )
            else:
                theme_list[counter][1] += p.get_text().strip().replace( '\n' , '.' )
        except:
            try:
                theme_list[counter][1] += p.get_text().strip().replace( '\n' , '.' )
            except:
                pass

    url_list[i].append(theme_list)
    #print("\r" + str(i).zfill(5)+"回目の処理を終えました！" , end="")
          
print("スクレイピングが終了しました。")

全部で287回処理を行います、


100%|██████████| 287/287 [17:09<00:00,  3.59s/it]

スクレイピングが終了しました。


In [14]:
#空のデータフレームをはじめに作って、そこにどんどん結合していく
import pandas as pd
df = pd.DataFrame()

for i in range(len(url_list)):
    df_tmp = pd.DataFrame(url_list[i][2],columns=["theme","text"])
    df_tmp["title"] = url_list[i][0]
    df_tmp["url"] = url_list[i][1]
    df = pd.concat([df,df_tmp])

df = df.reset_index()
df = df.drop(df.columns[[0]], axis=1)
df

,theme,text,title,url
0,授業研究：熱力学,授業研究：熱力学 長倉さんの発表. 長倉さんは 初めて高３物理を持つことになり、高校３年生の...,ＹＰＣ2021年07月例会(07/25 Zoomによるオンライン例会）,https://www2.hamajima.co.jp/~tenjin/ypc/ypc217...
1,ピップエレキバンとチョークコイルによる気体分子運動模型,ピップエレキバンとチョークコイルによる気体分子運動模型 天野さんの紹介. 天野さんが紹介して...,ＹＰＣ2021年07月例会(07/25 Zoomによるオンライン例会）,https://www2.hamajima.co.jp/~tenjin/ypc/ypc217...
2,Wordの書き込み式プリントに使える方眼紙を簡単に作る方法,Wordの書き込み式プリントに使える方眼紙を簡単に作る方法 櫻井さんの発表. 櫻井さんは、自...,ＹＰＣ2021年07月例会(07/25 Zoomによるオンライン例会）,https://www2.hamajima.co.jp/~tenjin/ypc/ypc217...
3,markdownがすごい!,markdownがすごい! 長倉さんの発表. 長倉さんは、授業プリントと授業用のスライドは、...,ＹＰＣ2021年07月例会(07/25 Zoomによるオンライン例会）,https://www2.hamajima.co.jp/~tenjin/ypc/ypc217...
4,二酸化炭素で気柱共鳴,二酸化炭素で気柱共鳴 長舩さんの発表. 水位を変える気柱共鳴実験器で媒質（管内の気体）を二酸...,ＹＰＣ2021年07月例会(07/25 Zoomによるオンライン例会）,https://www2.hamajima.co.jp/~tenjin/ypc/ypc217...
...,...,...,...,...
3771,飛び出す色環,飛び出す色環 喜多さんの発表例の立体メガネChromaDepth３Ｄで見ると中央が飛び出して...,ＹＰＣ1997年１１月例会（11/12柿生西高校）,https://www2.hamajima.co.jp/~tenjin/ypc/ypc97y...
3772,レーザー光線で音の振動波形を見る,レーザー光線で音の振動波形を見る右近さんが披露してくれた実験。ミラーキューブを空冷ファンに接...,ＹＰＣ1997年10月例会（10/22慶應義塾高校）,https://www2.hamajima.co.jp/~tenjin/ypc/ypc97x...
3773,ビー玉による圧力のモデル,ビー玉による圧力のモデル 水上さんの科学の祭典ステージ紹介上のレールから台はかりの上にビー...,ＹＰＣ1997年８月例会（8/30県立厚木高校）,https://www2.hamajima.co.jp/~tenjin/ypc/ypc978...
3774,手作りラブメータ（ハンドボイラ）,手作りラブメータ（ハンドボイラ） 高杉さん紹介資料瓶とエーテルで自作できます。..こちらは厚...,ＹＰＣ1997年８月例会（8/30県立厚木高校）,https://www2.hamajima.co.jp/~tenjin/ypc/ypc978...


In [15]:
#タイムスタンプをつけてcsv出力＆ダウンロード
import datetime
from google.colab import files
now = datetime.datetime.now()
filename = 'ypclog_' + now.strftime('%Y%m%d_%H%M%S') + '.csv'
print(filename)
df.to_csv(filename , encoding = 'utf-8-sig')
files.download(filename)

ypclog_20210920_130402.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>